### Case Study - program logic

- Read the file using the csv reader, create the shipments object and add all the rows in the shipments object
- Convert datetime into a datetime data type so it's not a string
- Then you need to calculate the number of missing product as a %, then the number of bad product also as a %... to each list, we will add two columns

In [1]:
import csv

In [34]:
# Create class
class Shipments:
    def __init__(self, date_received, ingredient, supplier, num_boxes, units_per_box, sample_count1\
            , sample_count2, sample_count3, rej_1, rej_2, rej_3):
        self.date_received = date_received
        self.ingredient = ingredient
        self.supplier = supplier
        self.num_boxes = int(num_boxes)
        self.units_per_box = int(units_per_box)
        self.sample_count1 = int(sample_count1)
        self.sample_count2 = int(sample_count2)
        self.sample_count3 = int(sample_count3)
        self.rej_1 = int(rej_1)
        self.rej_2 = int(rej_2)
        self.rej_3 = int(rej_3)
        self.total_rejections = (self.rej_1 + self.rej_2 + self.rej_3)/(self.units_per_box*self.num_boxes)*100
        self.all_data = (self.date_received, self.ingredient, self.supplier, self.num_boxes)
   


In [35]:
def read_shipments(file_name):
    global shipments # Make shipments global variable so we can access it
    shipments = []
    with open(file_name,'r') as f:
        reader = csv.reader(f)
        next(reader) # Skip the first row, which is our header
        for row in reader:
            date_received, ingredient, supplier, num_boxes, units_per_box, sample_count1\
            , sample_count2, sample_count3, rej_1, rej_2, rej_3 = row[0], row[1], row[2], row[3], row[4],row[5],row[6],row[7],row[8],row[9],row[10]
            shipments.append(Shipments(date_received, ingredient, supplier, num_boxes, units_per_box, sample_count1\
            , sample_count2, sample_count3, rej_1, rej_2, rej_3))
    return shipments   

In [36]:
read_shipments('/Users/delinaivanova/Downloads/3967_Module 5_Data File_rejections.csv')

In [38]:
shipments[0].all_data

('10-Jun-21', 'tomato_cherry', 'ABC Vegetables', 10)

In [48]:
for i in range(15):
    print(shipments[i].date_received, shipments[i].ingredient, shipments[i].supplier,shipments[i].total_rejections)


10-Jun-21 tomato_cherry ABC Vegetables 1.3
10-Jun-21 tomato_cherry Tomato Land 0.35000000000000003
10-Jun-21 tomato_vine Tomato Land 0.0
10-Jun-21 tomato_vine ABC Vegetables 0.0
10-Jun-21 cucumber_english ABC Vegetables 1.4285714285714286
10-Jun-21 cucumber_pickle Dill Farms 0.0
11-Jun-21 bananas_organic Banana Farms Inc 0.0
11-Jun-21 bananas_organic ABC Vegetables 0.38461538461538464
11-Jun-21 bananas_organic Banana Farms Inc 0.0
11-Jun-21 apples_gala Orchard Inc 0.5555555555555556
11-Jun-21 apples_gala Orchard Inc 0.6666666666666667
11-Jun-21 apples_gs Orchard Inc 8.75
13-Jun-21 apples_gala Apple Farm 0.0
13-Jun-21 oranges_california Cali Fruits 0.2222222222222222
13-Jun-21 oranges_california ABC Vegetables 0.075


In [51]:
consolidated_rejections = {}

for i in shipments:
    # create a unique key which captures the date + supplier
    
    if i.total_rejections > 0:
        key = (i.date_received, i.supplier)

        if key not in consolidated_rejections:
            consolidated_rejections[key] = []
    
        consolidated_rejections[key].append((i.ingredient, i.total_rejections))




In [50]:
consolidated_rejections

{('10-Jun-21', 'ABC Vegetables'): [('tomato_cherry', 1.3),
  ('tomato_vine', 0.0),
  ('cucumber_english', 1.4285714285714286)],
 ('10-Jun-21', 'Tomato Land'): [('tomato_cherry', 0.35000000000000003),
  ('tomato_vine', 0.0)],
 ('10-Jun-21', 'Dill Farms'): [('cucumber_pickle', 0.0)],
 ('11-Jun-21', 'Banana Farms Inc'): [('bananas_organic', 0.0),
  ('bananas_organic', 0.0)],
 ('11-Jun-21', 'ABC Vegetables'): [('bananas_organic', 0.38461538461538464)],
 ('11-Jun-21', 'Orchard Inc'): [('apples_gala', 0.5555555555555556),
  ('apples_gala', 0.6666666666666667),
  ('apples_gs', 8.75)],
 ('13-Jun-21', 'Apple Farm'): [('apples_gala', 0.0)],
 ('13-Jun-21', 'Cali Fruits'): [('oranges_california', 0.2222222222222222),
  ('grapes_red', 0.5714285714285714),
  ('grapes_green', 3.7142857142857144)],
 ('13-Jun-21', 'ABC Vegetables'): [('oranges_california', 0.075)]}

In [56]:
for (a,c),b in consolidated_rejections.items():
    print(a)
    print(c)
    print(b)

10-Jun-21
ABC Vegetables
[('tomato_cherry', 1.3), ('cucumber_english', 1.4285714285714286)]
10-Jun-21
Tomato Land
[('tomato_cherry', 0.35000000000000003)]
11-Jun-21
ABC Vegetables
[('bananas_organic', 0.38461538461538464)]
11-Jun-21
Orchard Inc
[('apples_gala', 0.5555555555555556), ('apples_gala', 0.6666666666666667), ('apples_gs', 8.75)]
13-Jun-21
Cali Fruits
[('oranges_california', 0.2222222222222222), ('grapes_red', 0.5714285714285714), ('grapes_green', 3.7142857142857144)]
13-Jun-21
ABC Vegetables
[('oranges_california', 0.075)]


In [57]:
# Write our files to send to suppliers

output_folder = '/Users/delinaivanova/Library/CloudStorage/OneDrive-Personal/UoT/Micro Courses/Python Programming for Automation/Emails_to_Suppliers'

In [58]:
for (date,supplier),rejections in consolidated_rejections.items():
    supplier_clean = supplier.replace(' ','_').replace('/','_')
    date_clean = date.replace('-','_')

    file_name = f"{supplier_clean}_{date_clean}.txt"
    file_path = f"{output_folder}/{file_name}"

    # add content of the file

    summary_lines = [
        f"Summary of rejected products for {supplier} on {date}\n",
        f"{'Product Code':<20}{'Quantity Rejected':<20}\n",
        f"{'-'*40}\n"]+[
            f"{product_code:<20}{quantity_rejected:<20}\n" for product_code, quantity_rejected in rejections
        ]
    
    with open(file_path,'w') as f:
        f.writelines(summary_lines)

    print(f"written_file: {file_path}")

written_file: /Users/delinaivanova/Library/CloudStorage/OneDrive-Personal/UoT/Micro Courses/Python Programming for Automation/Emails_to_Suppliers//ABC_Vegetables_10_Jun_21.txt
written_file: /Users/delinaivanova/Library/CloudStorage/OneDrive-Personal/UoT/Micro Courses/Python Programming for Automation/Emails_to_Suppliers//Tomato_Land_10_Jun_21.txt
written_file: /Users/delinaivanova/Library/CloudStorage/OneDrive-Personal/UoT/Micro Courses/Python Programming for Automation/Emails_to_Suppliers//ABC_Vegetables_11_Jun_21.txt
written_file: /Users/delinaivanova/Library/CloudStorage/OneDrive-Personal/UoT/Micro Courses/Python Programming for Automation/Emails_to_Suppliers//Orchard_Inc_11_Jun_21.txt
written_file: /Users/delinaivanova/Library/CloudStorage/OneDrive-Personal/UoT/Micro Courses/Python Programming for Automation/Emails_to_Suppliers//Cali_Fruits_13_Jun_21.txt
written_file: /Users/delinaivanova/Library/CloudStorage/OneDrive-Personal/UoT/Micro Courses/Python Programming for Automation/Ema

In [27]:
with open('/Users/delinaivanova/Downloads/3967_Module 5_Data File_rejections.csv','r') as f:
    reader = csv.reader(f)
    for row in reader:
        print(', '.join(row))

date_received, ingredient_name, supplier, number_of_cases, units_per_case, sample_case_count1, sample_case_count2, sample_case_count3, sample_case_rejections1, sample_case_rejections2, sample_case_rejections3
10-Jun-21, tomato_cherry, ABC Vegetables, 10, 100, 95, 100, 110, 5, 3, 5
10-Jun-21, tomato_cherry, Tomato Land, 20, 500, 440, 480, 500, 10, 0, 25
10-Jun-21, tomato_vine, Tomato Land, 25, 100, 100, 100, 100, 0, 0, 0
10-Jun-21, tomato_vine, ABC Vegetables, 10, 100, 100, 98, 101, 0, 0, 0
10-Jun-21, cucumber_english, ABC Vegetables, 14, 50, 51, 50, 51, 10, 0, 0
10-Jun-21, cucumber_pickle, Dill Farms, 15, 50, 50, 50, 50, 0, 0, 0
11-Jun-21, bananas_organic, Banana Farms Inc, 24, 40, 40, 40, 40, 0, 0, 0
11-Jun-21, bananas_organic, ABC Vegetables, 26, 30, 30, 28, 29, 0, 3, 0
11-Jun-21, bananas_organic, Banana Farms Inc, 20, 30, 30, 30, 30, 0, 0, 0
11-Jun-21, apples_gala, Orchard Inc, 20, 90, 90, 94, 90, 1, 4, 5
11-Jun-21, apples_gala, Orchard Inc, 5, 90, 90, 90, 90, 0, 0, 3
11-Jun-21, app